In [1]:

%pprint
%matplotlib inline
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF



---
# Load needed libraries and functions

In [2]:

from IPython.display import clear_output
from jobpostlib import (duration, freq, t0, nu, hau, wsu, cu, ihu, hc, lru, slrcu, ssgdcu, scrfcu, crf, su, time, humanize, winsound)
from os import path as osp
from pandas import DataFrame
import numpy as np
import os
import re
import sys
import warnings

warnings.filterwarnings('ignore')

======== Neo4j/4.4.7 ========
Utility libraries created in 14 seconds


In [3]:

# Parts-of-speech CRF elements built in 28 minutes and 51 seconds
t1 = time.time()
if not (hasattr(scrfcu, 'pos_symbol_crf') or crf.is_flask_running()):
    scrfcu.build_pos_conditional_random_field_elements(verbose=True)
if hasattr(scrfcu, 'pos_symbol_crf'):
    print('predict_single is now available')
else:
    print('predict_single is not available')
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Parts-of-speech conditional random field elements built in {duration_str}')

predict_single is now available
Parts-of-speech conditional random field elements built in 6 seconds


In [4]:

# Parts-of-speech stochastic gradient decent elements built in 17 seconds
t1 = time.time()
if not (hasattr(ssgdcu, 'pos_predict_percent_fit_dict') or crf.is_flask_running()):
    ssgdcu.build_pos_stochastic_gradient_descent_elements(sampling_strategy_limit=None, verbose=True)
if hasattr(ssgdcu, 'pos_predict_percent_fit_dict'):
    print('predict_single is now available')
else:
    print('predict_single is not available')
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Parts-of-speech stochastic gradient decent elements built in {duration_str}')

I have 49,102 labeled parts of speech in here
predict_single is now available
Parts-of-speech stochastic gradient decent elements built in 17 seconds


In [5]:

# Check if the crf has built its parts-of-speech classifier
# POS classifier trained in 12 hours, 15 minutes and 36 seconds
t1 = time.time()
if not hasattr(crf, 'CRF'):
    if nu.pickle_exists('crf_CRF'):
        crf.CRF = nu.load_object('crf_CRF')
    else:
        crf.retrain_pos_classifier(header_pattern_dict=nu.load_object('HEADER_PATTERN_DICT'), verbose=True)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'POS classifier trained in {duration_str}')

POS classifier trained in 0 seconds


In [6]:

# Parts-of-speech logistic regression elements built in 1 hour, 59 minutes and 41 seconds
t1 = time.time()
if not (hasattr(slrcu, 'pos_predict_percent_fit_dict') or crf.is_flask_running()):
    slrcu.build_pos_logistic_regression_elements(sampling_strategy_limit=None, verbose=True)
if hasattr(slrcu, 'pos_predict_percent_fit_dict'):
    print('predict_single is now available')
else:
    print('predict_single is not available')
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Parts-of-speech logistic regression elements built in {duration_str}')

I have 49,102 labeled parts of speech in here
predict_single is now available
Parts-of-speech logistic regression elements built in 13 seconds


In [7]:

# Is-qualified LR classifier built in 5 seconds
t1 = time.time()
lru.build_isqualified_logistic_regression_elements(sampling_strategy_limit=5_000, verbose=False)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Is-qualified LR classifier built in {duration_str}')

Is-qualified LR classifier built in 9 seconds


In [8]:

# Is-header SGD classifer built in 9 seconds
t1 = time.time()
ihu.build_pos_stochastic_gradient_descent_elements(verbose=True)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Is-header SGD classifer built in {duration_str}')

I have 49,856 hand-labeled header htmls prepared
7 iterations seen during training fit for a total of 49,856 records trained
Is-header SGD classifer built in 9 seconds



---
# Training

In [53]:

# You need to run this again if you changed the qualification dictionary in another notebook
t1 = time.time()

# Keep the total retraining time to less than two minutes by adjusting the sampling strategy limit
lru.sync_basic_quals_dict(sampling_strategy_limit=8_000, verbose=False)
lru.retrain_isqualified_classifier(verbose=True)

duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Is-qualified classifer retrained in {duration_str}')

I have 14,583 hand-labeled qualification strings in here
I have 484,937 is-qualified vocabulary tokens in here
Is-qualified classifer retrained in 2 minutes and 3 seconds


In [54]:

cypher_str = f'''
    MATCH (fn:FileNames)
    WHERE
        fn.percent_fit = 0.0 AND
        ((fn.is_closed IS NULL) OR (fn.is_closed = false)) AND
        ((fn.is_verified IS NULL) OR (fn.is_verified = false)) AND
        ((fn.is_opportunity_application_emailed IS NULL) OR
        (fn.is_opportunity_application_emailed = false))
    RETURN
        fn.percent_fit AS percent_fit,
        fn.file_name AS file_name,
        fn.posting_url AS url
    ORDER BY fn.percent_fit ASC;'''
row_objs_list = []
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
files_list = []
if row_objs_list:
    files_list = DataFrame(row_objs_list).file_name.tolist()
print(f'Only {len(files_list)} more mis-estimated minimum-requirements-met percentages to go!')

Only 0 more mis-estimated minimum-requirements-met percentages to go!


In [79]:

cypher_str = f'''
    MATCH (fn:FileNames)
    WHERE
        (fn.percent_fit >= 0.8) AND
        ((fn.is_closed IS NULL) OR (fn.is_closed = false)) AND
        ((fn.is_verified IS NULL) OR (fn.is_verified = false)) AND
        (
            (fn.is_opportunity_application_emailed IS NULL) OR
            (fn.is_opportunity_application_emailed = false)
        )
    RETURN
        fn.percent_fit AS percent_fit,
        fn.file_name AS file_name,
        fn.posting_url AS posting_url
    ORDER BY fn.percent_fit DESC;'''
row_objs_list = []
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
files_list = []
if row_objs_list:
    files_list = DataFrame(row_objs_list).file_name.tolist()
print(f'Only {len(files_list)} more minimum-requirements-met percentages to confirm!')

Only 0 more minimum-requirements-met percentages to confirm!



----
## Fix POS and Quals for a job posting

In [80]:

t1 = time.time()
file_name = files_list.pop()
file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
if os.path.isfile(file_path):
    child_strs_list = hau.get_child_strs_from_file(file_name=file_name)
    cu.ensure_filename(file_name, verbose=False)
    cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
    child_tags_list = hau.get_child_tags_list(child_strs_list)
    feature_dict_list = cu.get_feature_dict_list(child_tags_list, child_strs_list)
    feature_tuple_list = []
    for feature_dict in feature_dict_list:
        feature_tuple_list.append(hc.get_feature_tuple(
            feature_dict, pos_lr_predict_single=slrcu.predict_single, pos_crf_predict_single=scrfcu.predict_single,
            pos_sgd_predict_single=ssgdcu.predict_single
        ))
    crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
    print(file_name)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'CRF and child strings list recreated in {duration_str}')

IndexError: pop from empty list

In [58]:

db_pos_list = []
for navigable_parent in child_strs_list:
    db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
pos_list, indices_list = su.visualize_basic_quals_section(crf_list, child_strs_list, db_pos_list=db_pos_list, verbose=True)
winsound.Beep(freq, duration)

['H-IP', 'O-IP', 'H-IP', 'O-IP', 'O-IP', 'O-IP', 'O-IP', 'O-IP', 'O-IP', 'O-IP', 'O-IP', 'O-IP', 'O-IP', 'H-JD', 'O-SP', 'H-JD', 'O-SP', 'O-IP', 'O-OL', 'O-OL', 'O-IP', 'H-SP', 'O-IP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'H-O', 'O-O', 'O-IP', 'H-O', 'H-JT', 'H-JT', 'O-JD', 'O-JD', 'O-JD', 'O-JD', 'O-CS', 'H-TS', 'O-TS', 'H-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'H-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-PQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-PQ', 'O-RQ', 'H-PQ', 'O-RQ', 'H-RQ', 'O-RQ', 'H-TS', 'O-JT', 'O-JT', 'O-OL', 'H-SP', 'O-SP', 'H-SP', 'H-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-SP', 'O-IP', 'O-IP', 'O-IP', 'O-LN', 'O-IP', 'O-IP']
[59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 73, 77]


[59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 73, 77]


In [ ]:
raise

In [60]:

basic_quals_dict = nu.load_object('basic_quals_dict'); column_name = 'is_minimum_qualification'
for idx in list(range(58, 74)) + list(range(76, 78)):
    child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]
    def do_cypher_tx(tx, navigable_parent, verbose=False):
        cypher_str = '''
            MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
            ''' + cu.return_everything_str + ';'
        results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})
        return [dict(record.items()) for record in results_list]
    with cu.driver.session() as session: row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=False)
    print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end='')
    print(f'{idx} {pos_symbol}) {child_str}')
    def do_cypher_tx(tx, navigable_parent, verbose=False):
        cypher_str = f'''
            MATCH (np:NavigableParents {{navigable_parent: $navigable_parent}})
            SET
                np.is_task_scope = {str(column_name == 'is_task_scope').lower()},
                np.is_minimum_qualification = {str(column_name == 'is_minimum_qualification').lower()},
                np.is_preferred_qualification = {str(column_name == 'is_preferred_qualification').lower()},
                np.is_educational_requirement = {str(column_name == 'is_educational_requirement').lower()},
                np.is_legal_notification = {str(column_name == 'is_legal_notification').lower()},
                np.is_other = {str(column_name == 'is_other').lower()},
                np.is_corporate_scope = {str(column_name == 'is_corporate_scope').lower()},
                np.is_job_title = {str(column_name == 'is_job_title').lower()},
                np.is_office_location = {str(column_name == 'is_office_location').lower()},
                np.is_job_duration = {str(column_name == 'is_job_duration').lower()},
                np.is_supplemental_pay = {str(column_name == 'is_supplemental_pay').lower()},
                np.is_interview_procedure = {str(column_name == 'is_interview_procedure').lower()},
                np.is_posting_date = {str(column_name == 'is_posting_date').lower()}
            ''' + cu.return_everything_str + ';'
        return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})]
    with cu.driver.session() as session: row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=False)

58 H-RQ) <b>Required Qualifications:</b>
1
59 O-RQ) <li>Undergraduate or a postgraduate degree in Mathematics, Statistics, Computer Science, Economics, Data Science, or a related field.</li>
60 O-RQ) <li>At least three (3) years of experience in Data Science with a focus on A/B testing, model tracking, and ML engineering.</li>
1
61 O-RQ) <li>Strong experience in programming languages such as SQL, Python.</li>
62 O-RQ) <li>Experience with AWS cloud services, especially those related to machine learning (e.g., S3, EC2, Lambda, SageMaker).</li>
63 O-RQ) <li>Working knowledge of machine learning frameworks (e.g., TensorFlow, PyTorch) and libraries (e.g., scikit-learn, Pandas).</li>
1
64 O-RQ) <li>Advanced abilities to handle large datasets.</li>
1
65 O-RQ) <li>Results-oriented individual with the ability to translate plans into actions.</li>
1
66 O-RQ) <li>Ability to work in a fast-paced environment; ability to multi-task, change direction, effectively prioritize, and meet deadlines with b

In [75]:

# Display the context of an individual child string
idx = 77
print(indices_list); child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]; basic_quals_dict = nu.load_object('basic_quals_dict')
print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end=''); print(f'{idx} {pos_symbol}) {child_str}')

[59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 73, 77]
77 O-RQ) While performing the duties of this job, the employee is frequently required to sit for extended periods; reach with hands and arms; and talk or hear. The employee is occasionally required to move about. The employee must occasionally lift and/or move up to twenty (20) pounds. Specific vision abilities required by this job include close vision and the ability to adjust focus.


In [76]:

# Hand-label this particular child string in the quals dictionary
basic_quals_dict = nu.load_object('basic_quals_dict')
basic_quals_dict[child_str] = 1
nu.store_objects(basic_quals_dict=basic_quals_dict); print(f'"{child_str}" in basic_quals_dict: {basic_quals_dict[child_str]}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"While performing the duties of this job, the employee is frequently required to sit for extended periods; reach with hands and arms; and talk or hear. The employee is occasionally required to move about. The employee must occasionally lift and/or move up to twenty (20) pounds. Specific vision abilities required by this job include close vision and the ability to adjust focus." in basic_quals_dict: 1



----
## Maintenance

In [78]:

# Mark the file name as needing retraining everywhere
# file_name = 'b4e994e1d282ffa9_Digital_Platform_Services_Data_Analytics_and_Insights_Senior_Manager_Salt_Lake_City_UT_84111_Indeed_com.html'
mask_series = lru.hunting_df.file_name.isin([file_name])
lru.hunting_df.loc[mask_series, 'percent_fit'] = np.nan
nu.store_objects(hunting_df=lru.hunting_df)
def do_cypher_tx(tx, file_name, verbose=False):
    cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.percent_fit = NULL, fn.is_verified = true
        RETURN fn;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$file_name', f'"{file_name}"'))
    results_list = tx.run(query=cypher_str, parameters={'file_name': file_name})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, file_name=file_name, verbose=False)
row_objs_list

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\hunting_df.pkl


[{'fn': <Node element_id='1006088' labels=frozenset({'FileNames'}) properties={'file_name': 'Data_Scientist_II_Remote_Indeed_com.html', 'posting_url': 'https://www.indeed.com/pagead/clk/dl?mo=r&ad=-6NYlbfkN0CRXJaX_ETJGlwN1sk8fjXo5yVXRvSeounu7t0bSIDpPtRyf9H7S5sByoi2jQ_0fiPvAWntulR0LmDy-Xg99bgMzdM7vjX7z2wUOGVcbckTanAEPHtlpW0EHlxxWyMHD_gQQcW0yrrv9YkqSt5OzkF9AwO5ocrAcO9Yslr0DzST3-PewVYSFWC59AutsQYwhD-kJZYd_2HsVgi-YKa2NnamR1PPp2sWw17ExQnYpicbrSwFA_YbIGQo7d196ogHDtl4v6UT36LORsf5lIy26gT-wZRnLMz2vyzH01yAHLDHI60GDz4ta9_eI0OHFOwtl0EToLKyMdxtMlvMb_rVbLVH7DLU4xjecRO8YF5DA0gWUKekGd5DA8bvhnaWZPcZrgvF4dIoMdOhtcPJAGC3vMxqv8rWN4QdiFcnNWW_iFSWf_rn6AZL-9Mi9MHeJ5U6_jFCxg6U_F_TExgaJWzfaBJnkzmqK-L7-KMzyKxkky6B9BTaAkcd0quJ5ZTQBtoB4C2u3tINWfI_wekgskmZs-TE3u2N4UshDDIn4gsIQ0V71qjKMqg5BeB0XuwFVMiwxgVl_Rur2oekfpMduLOwbhf-knaOoSXReIuB3YOTtOct--tuHd9f0KcdHHd73eMRObIuJFVWg-y2ZFUcm4FTOIZuKl-lj6G9GK-NKFSXDmKLWk6YRcchQyZsCZ1tF7s57sjMvVF8uKFIjr3CwOVX5Z7DIZORDDYdTYW4NjADZW0vCfCwDw0XWyZY4bivACkNle1em5inCN3H1kMMzIFocOH5-_2

In [77]:

# Fix preferred quals
basic_quals_dict = nu.load_object('basic_quals_dict')
for idx in list(range(58, 74)) + list(range(76, 78)):
    child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]
    def do_cypher_tx(tx, navigable_parent, verbose=False):
        cypher_str = '''
            MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
            ''' + cu.return_everything_str + ';'
        results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})
        return [dict(record.items()) for record in results_list]
    with cu.driver.session() as session: row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=False)
    if re.search(r'\b(nice(er)?|a plus|advantageous|preferred|beneficial)\b', child_str):
        print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end='')
        print(f'{idx} {pos_symbol}) {child_str}')
        def do_cypher_tx(tx, navigable_parent, verbose=False):
            cypher_str = f'''
                MATCH (np:NavigableParents {{navigable_parent: $navigable_parent}})
                SET
                    np.is_minimum_qualification = false,
                    np.is_preferred_qualification = true
                ''' + cu.return_everything_str + ';'
            return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})]
        with cu.driver.session() as session: row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=False)

In [40]:

# Fix Headers
basic_quals_dict = nu.load_object('basic_quals_dict')
for idx in [40, 49, 61]:
    child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]
    print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end='')
    print(f'{idx} {pos_symbol}) {child_str}')
    def do_cypher_tx(tx, navigable_parent, verbose=False):
        cypher_str = f'''
            MATCH (np:NavigableParents {{navigable_parent: $navigable_parent}})
            SET
                np.is_header = true
            ''' + cu.return_everything_str + ';'
        return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})]
    with cu.driver.session() as session: row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=False)

40 H-TS) <p>Required Qualifications:</p>
49 O-TS) <p>Skills and Abilities:</p>
61 H-TS) <p>Experience:</p>


In [31]:

# Fix Non-headers
basic_quals_dict = nu.load_object('basic_quals_dict')
for idx in [75]:
    child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]
    print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end='')
    print(f'{idx} {pos_symbol}) {child_str}')
    def do_cypher_tx(tx, navigable_parent, verbose=False):
        cypher_str = f'''
            MATCH (np:NavigableParents {{navigable_parent: $navigable_parent}})
            SET
                np.is_header = false
            ''' + cu.return_everything_str + ';'
        return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})]
    with cu.driver.session() as session: row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=False)

1
75 H-RQ) <div>
     Education: Master’s degree in Statistics, Data Science, Data Analytics, Health Informatics, or related field of study
   </div>


In [73]:

# Break up overly-long O-RQs:
# Ensure you have already displayed the context of an individual child string above
# Don't close the Notepad++ window until you have replaced the child string
def display_file_in_text_editor(file_name):
    text_editor_path = r'C:\Program Files\Notepad++\notepad++.exe'
    file_path = osp.abspath(osp.join(hau.SAVES_HTML_FOLDER, file_name))
    !"{text_editor_path}" "{file_path}"
display_file_in_text_editor(file_name)
cu.rebuild_filename_node(file_name, navigable_parent=None, verbose=True)
winsound.Beep(freq, duration)


                MATCH
                    (np:NavigableParents {navigable_parent: "is proud to be an equal opportunity workplace and is an affirmative action employer. We invite you to join us and feel the VIBES: https://vibronyx.com/about/"}),
                    (ht:HeaderTags {header_tag: "plaintext"})
                MERGE (ht)-[r:SUMMARIZES]->(np);


In [148]:

# Remove this particular child string from the quals dictionary and database
basic_quals_dict = nu.load_object('basic_quals_dict')
child_str = child_strs_list[idx]
basic_quals_dict.pop(child_str, None)
# basic_quals_dict[child_str] = 0
nu.store_objects(basic_quals_dict=basic_quals_dict)
print(f'"{child_str}" in basic_quals_dict: {child_str in basic_quals_dict}')
def do_cypher_tx(tx, qualification_str, verbose=False):
    cypher_str = '''
        MATCH (qs:QualificationStrings {qualification_str: $qualification_str})
        DETACH DELETE qs;
        '''
    results_list = tx.run(query=cypher_str, parameters={'qualification_str': qualification_str})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, qualification_str=child_str, verbose=False)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"<i>at least 5-7 years’ experience as a site </i>" in basic_quals_dict: False



----

In [28]:

def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = """MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
        SET
            np.is_header = false,
            np.is_task_scope = false,
            np.is_minimum_qualification = true,
            np.is_preferred_qualification = false,
            np.is_educational_requirement = false,
            np.is_legal_notification = false,
            np.is_other = false,
            np.is_corporate_scope = false,
            np.is_job_title = false,
            np.is_office_location = false,
            np.is_job_duration = false,
            np.is_supplemental_pay = false,
            np.is_interview_procedure = false,
            np.is_posting_date = false
        """ + cu.return_everything_str + ';'
    return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
ihu.retrain_classifier(row_objs_list[0]['navigable_parent'], row_objs_list[0]['is_header'], verbose=True)
row_objs_list

1 iterations seen during updating fit for a total of 49,070 records trained


[{'navigable_parent': '<p>Selected candidate will be subject to a pre-employment background investigation and must be able to obtain and maintain a Secret level DoD security clearance.</p>', 'is_header': False, 'is_task_scope': False, 'is_minimum_qualification': True, 'is_preferred_qualification': False, 'is_legal_notification': False, 'is_job_title': False, 'is_office_location': False, 'is_job_duration': False, 'is_supplemental_pay': False, 'is_educational_requirement': False, 'is_interview_procedure': False, 'is_corporate_scope': False, 'is_posting_date': False, 'is_other': False}]

In [83]:

# Show what's in the database already for this html string
def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = '''MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
        ''' + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
row_objs_list

[{'navigable_parent': '<b>Who makes a DatOps Engineer?</b>', 'is_header': None, 'is_task_scope': None, 'is_minimum_qualification': None, 'is_preferred_qualification': None, 'is_legal_notification': None, 'is_job_title': None, 'is_office_location': None, 'is_job_duration': None, 'is_supplemental_pay': None, 'is_educational_requirement': None, 'is_interview_procedure': None, 'is_corporate_scope': None, 'is_posting_date': None, 'is_other': None}]

In [335]:

lru.sync_basic_quals_dict(sampling_strategy_limit=8_000, verbose=False)
df = lru.basic_quals_df.copy()
df['length'] = df['qualification_str'].apply(len)
df = df.sort_values('length')
df.to_csv('../saves/csv/basic_quals_dict.csv')
df.head(1000).tail(5)

,qualification_str,is_qualified,length
1353,<li>Hadoop and Machine Learning</li>,0,36
1573,<p>· Experience with API testing</p>,0,36
4903,<orq>3 - MySQL (P3 - Advanced)</orq>,0,36
2823,<li>An understanding of rating.</li>,0,36
6492,<orq>1 - MySQL (P1 - Beginner)</orq>,1,36


In [330]:

# Remove the child string by idx from the quals dictionary and database
basic_quals_dict = nu.load_object('basic_quals_dict')
child_str = df.loc[13195].qualification_str
basic_quals_dict.pop(child_str, None)
nu.store_objects(basic_quals_dict=basic_quals_dict)
print(f'"{child_str}" in basic_quals_dict: {child_str in basic_quals_dict}')
def do_cypher_tx(tx, qualification_str, verbose=False):
    cypher_str = '''
        MATCH (qs:QualificationStrings {qualification_str: $qualification_str})
        DETACH DELETE qs;
        '''
    results_list = tx.run(query=cypher_str, parameters={'qualification_str': qualification_str})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, qualification_str=child_str, verbose=False)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"<orq>Ability to nice to have).</orq>" in basic_quals_dict: False


In [ ]:

# You've made no changes to the qualification dictionary (regardless of parts-of-speech changes)
def do_cypher_tx(tx, file_name, verbose=False):
    cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.is_verified = true
        RETURN fn;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$file_name', f'"{file_name}"'))
    parameter_dict = {'file_name': file_name}
    results_list = tx.run(query=cypher_str, parameters=parameter_dict)
    values_list = []
    for record in results_list:
        values_list.append(dict(record.items()))

    return values_list
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, file_name=file_name, verbose=True)
row_objs_list

In [ ]:

# Mark the file name as closed
cypher_str = f'''
    MATCH (fn:FileNames {{file_name: "{file_name}"}})
    SET fn.is_closed = true
    RETURN fn;'''
print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

In [ ]:

# Manually label the unscored qual
qualification_str = quals_list[13]
print(qualification_str)
basic_quals_dict = nu.load_object('basic_quals_dict')
basic_quals_dict[qualification_str] = 0
nu.store_objects(basic_quals_dict=basic_quals_dict)

In [ ]:

# Remove file name from database
# file_name = '3c031ea6ad293e92_General_Service_Technician_Westborough_MA_01581_Indeed_com.html'
cu.delete_filename_node(file_name, verbose=True)

In [ ]:

# Fix the unhashable mess you made
lru.basic_quals_df = lru.basic_quals_df.iloc[:-2]
nu.store_objects(basic_quals_df=lru.basic_quals_df)
lru.basic_quals_dict = lru.basic_quals_df.set_index(
    'qualification_str'
).is_qualified.to_dict()
lru.nu.store_objects(basic_quals_dict=lru.basic_quals_dict, verbose=True)

In [ ]:

# Fix the unhashable mess you made
def do_cypher_tx(tx, verbose=False):
    cypher_str = '''
        MATCH (qs:QualificationStrings)
        WHERE NOT qs.is_qualified IN [0, 1]
        DETACH DELETE qs;
        '''
    results_list = tx.run(query=cypher_str, parameters={})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, verbose=False)
cypher_str = '''
    MATCH (qs:QualificationStrings)
    RETURN qs;'''
row_objs_list = cu.get_execution_results(cypher_str, verbose=False)
DataFrame(
    [{k: v for k, v in row_obj['qs'].items()} for row_obj in row_objs_list]
).tail(5)


----
# Take a badly written requirements section and see if you can programmatically parse the qualification string out of it

In [16]:

def combine_adjacent(split_strs_list):
    combined_list = []
    for i, s in enumerate(split_strs_list):
        if i == 0:
            combined_list.append(s)
        elif combined_list[-1].lower().endswith(' and'):
            combined_list[-1] = combined_list[-1] + ' ' + s
        else:
            combined_list.append(s)
    
    return combined_list

In [17]:

# Break the long HTML string into sentences and check if each is a qualification string
t1 = time.time()
from langchain.text_splitter import SpacyTextSplitter
import re

fake_stops_list = ['e.g.', 'etc.', 'M.nu.', 'B.nu.', 'Ph.D.', '(ex.', '(Ex.',
                   'U.nu.', 'i.e.', '&amp;', 'E.g.', 'Bsc.', 'MSc.', 'incl.']
replacements_list = ['eg', 'etc', 'MS', 'BS', 'PhD', '(eg', '(eg', 'US',
                     'ie', '&', 'eg', 'BS', 'MS', 'include']
text_splitter = SpacyTextSplitter()
tag_regex = re.compile('<([a-z][a-z0-9]*)[^<>]*>')
rows_list = []
unhtml_str = re.sub('</?[^><]+>', '', child_str)
for fake_stop, replacement in zip(fake_stops_list, replacements_list):
    unhtml_str = unhtml_str.replace(fake_stop, replacement)
split_strs_list = combine_adjacent([str(split_str) for split_str in text_splitter._tokenizer(unhtml_str).sents])
for split_str in split_strs_list:
    row_dict = {}
    split_str = re.sub(r'\s*[:;.*]+\s*$', '', split_str)
    row_dict['split_str'] = split_str
    row_dict['char_count'] = len(split_str)
    match_obj = tag_regex.search(child_str)
    if match_obj:
        tag_name = match_obj.group()
        split_str = f'<{tag_name}>{split_str}</{tag_name}>'
    else:
        tag_name = 'plaintext'
    row_dict['tag_name'] = tag_name
    score = 1.0
    score *= slrcu.pos_predict_percent_fit_dict['O-RQ'](split_str)
    score *= scrfcu.pos_predict_percent_fit_dict['O-RQ'](split_str)
    score *= ssgdcu.pos_predict_percent_fit_dict['O-RQ'](split_str)
    row_dict['orq_score'] = score
    score = 1.0
    score *= slrcu.pos_predict_percent_fit_dict['O-PQ'](split_str)
    score *= scrfcu.pos_predict_percent_fit_dict['O-PQ'](split_str)
    score *= ssgdcu.pos_predict_percent_fit_dict['O-PQ'](split_str)
    row_dict['opq_score'] = score
    rows_list.append(row_dict)
split_orqs_df = DataFrame(rows_list)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Split O-RQs DataFrame built in {duration_str}')

Split O-RQs DataFrame built in 2 seconds


In [19]:

split_orqs_df.sort_values('orq_score', ascending=False)

,split_str,char_count,tag_name,orq_score,opq_score
23,3-5 years of experience in data science and da...,78,plaintext,3.818604e-01,1.237190e-06
26,Experience using AI/ML python libraries –PyTor...,259,plaintext,3.729124e-01,1.369416e-03
27,Experience working with MLOps infrastructure s...,128,plaintext,3.484962e-01,1.842955e-04
25,Proficient in Python programming,32,plaintext,2.616722e-01,1.276113e-08
10,This position requires a motivated individual ...,280,plaintext,1.635554e-01,1.455531e-07
28,Legally authorized to work in the US,36,plaintext,7.631722e-02,1.789053e-06
24,Knowledge and experience in some of these: Dee...,219,plaintext,6.289732e-02,4.190706e-04
16,"Architect, develop, deploy, and maintain scala...",219,plaintext,5.318624e-02,3.283486e-10
18,Work with cross-functional teams to derive the...,159,plaintext,4.171702e-02,9.168556e-09
21,Develop tools in analyzing diverse sets of imp...,222,plaintext,3.348758e-02,7.542002e-11


In [ ]:

# Take a badly written requirements section and see if you can programmatically parse the qualification string out of it
import re
from nltk.tokenize import sent_tokenize

# sampling_strategy_limit=6_400 gets 10,635 labeled parts of speech and takes 49 minutes and 30 seconds
# sampling_strategy_limit=7_000 gets 10,635 labeled parts of speech and takes 49 minutes and 30 seconds
slrcu.build_pos_logistic_regression_elements(sampling_strategy_limit=70_000, verbose=True)

qual_paragraph = re.sub('</?[^<>]+>', '', child_str.strip(), 0, re.MULTILINE)
if len(sent_tokenize(qual_paragraph)) < 2:
    child_strs_list = re.split(' *: *', qual_paragraph, 0)
    child_tags_list = hau.get_child_tags_list(child_strs_list)
    feature_dict_list = cu.get_feature_dict_list(child_tags_list, child_strs_list)
    feature_tuple_list = []
    for feature_dict in feature_dict_list:
        feature_tuple_list.append(hc.get_feature_tuple(feature_dict, pos_lr_predict_single=slrcu.predict_single, pos_crf_predict_single=None, pos_sgd_predict_single=None))
    crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
    if crf_list[0] == 'H-RQ':
        child_strs_list = re.split(' *; *', ': '.join(child_strs_list[1:]), 0)
        child_tags_list = hau.get_child_tags_list(child_strs_list)
        feature_dict_list = cu.get_feature_dict_list(child_tags_list, child_strs_list)
        feature_tuple_list = []
        for feature_dict in feature_dict_list:
            feature_tuple_list.append(hc.get_feature_tuple(feature_dict, pos_lr_predict_single=slrcu.predict_single, pos_crf_predict_single=None, pos_sgd_predict_single=None))
        crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
        db_pos_list = []
        for navigable_parent in child_strs_list:
            db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
        pos_list, indices_list = su.visualize_basic_quals_section(crf_list, child_strs_list, db_pos_list=db_pos_list, verbose=True)

In [ ]:

import nltk.tokenize

dir(nltk.tokenize)

In [ ]:

[f'nltk.tokenize.{fn}' for fn in dir(nltk.tokenize) if 'Tokenize' in fn]

In [ ]:

nltk.tokenize.TweetTokenizer?

In [ ]:

[f'nltk.tokenize.{fn}' for fn in dir(nltk.tokenize) if 'tokenize' in fn]

In [ ]:

nltk.tokenize.wordpunct_tokenize(child_str)

In [ ]:

nltk.tokenize.word_tokenize(child_str, preserve_line=True)

In [ ]:

list(nltk.tokenize.string_span_tokenize(child_str, r';\s*'))

In [ ]:

nltk.tokenize.sent_tokenize(child_str)

In [ ]:

list(nltk.tokenize.regexp_tokenize(child_str, r'\w+'))

In [ ]:

list(nltk.tokenize.regexp_span_tokenize(child_str, r'\s\s+'))

In [ ]:

nltk.tokenize.line_tokenize(child_str)

In [ ]:

nltk.tokenize.casual_tokenize(child_str)

In [ ]:

nltk.tokenize.blankline_tokenize(child_str)